add aqui danilove

In [ ]:
import pandas as pd
df_final = pd.read_csv("../dados/df_limpo.csv")

In [3]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from typing import List, Dict, Any
import json 
import re

df = df_final.head(200).copy()

#print(df['setor_economico'].value_counts())

# Carrega variáveis do .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [4]:
def criar_agrupamentos_setores(setores_unicos: List[str], num_grupos: int = 5) -> Dict[str, List[str]]:
    """
    Cria agrupamentos inteligentes de setores econômicos usando Gemini
    """
    prompt = f"""
    Analise os seguintes setores econômicos e crie {num_grupos} agrupamentos lógicos baseados em similaridade de atividade, cadeia produtiva ou segmento de mercado:

    Setores: {', '.join(setores_unicos)}

    Instruções:
    1. Crie {num_grupos} grupos distintos
    2. Cada grupo deve ter um nome descritivo com até 4 palavras
    3. Agrupe setores relacionados por atividade econômica similar
    4. Retorne no formato JSON estruturado

    Formato de resposta (JSON):
    {{
        "grupo_1": {{
            "nome": "Nome do Grupo",
            "setores": ["setor1", "setor2"]
        }},
        "grupo_2": {{
            "nome": "Nome do Grupo",
            "setores": ["setor3", "setor4"]
        }}
    }}
    """
    try:
        response = model.generate_content(prompt)
        response_text = response.text.strip()

        # Limpa o início e o fim se vierem marcadores de bloco de código
        if response_text.startswith("```json"):
            response_text = response_text[len("```json"):].strip()
        elif response_text.startswith("```"):
            response_text = response_text[len("```"):].strip()
        if response_text.endswith("```"):
            response_text = response_text[:-3].strip()

        # Remove vírgulas antes de fechar colchetes ou chaves
        response_text = re.sub(r",\s*([}$$])", r"\1", response_text)
            
        agrupamentos = json.loads(response_text)
        return agrupamentos
    except Exception as e:
        print(f"Erro ao criar agrupamentos: {e}")
        return {}


In [5]:
setores_unicos = df['setor_economico'].dropna().unique().tolist()

#agrupamentos = criar_agrupamentos_setores(setores_unicos, 15)

In [6]:
print(df['setor_economico'].dropna().unique().tolist())

['Fabricação de Peças e Acessórios para o Sistema Motor de Veículos Automotores', 'Gestão e Administração da Propriedade Imobiliária', 'Comércio Atacadista de Combustíveis Sólidos, Líquidos e Gasosos, Exceto Gás Natural e Glp', 'Construção de Edifícios# Construção de Rodovias e Ferrovias', 'Comércio de Peças e Acessórios para Veículos Automotores', 'Fabricação de Aparelhos Eletromédicos e Eletroterapêuticos e Equipamentos de Irradiação', 'Comércio Varejista de Gás Liqüefeito de Petróleo (Glp)', 'Comércio Varejista de Combustíveis para Veículos Automotores', 'Fabricação de Aparelhos e Equipamentos de Medida, Teste e Controle', 'Serviços de Engenharia', 'Fabricação de Resinas Termofixas# Fabricação de Resinas Termoplásticas', 'Fabricação de Embalagens de Material Plástico', 'Aluguel de Máquinas e Equipamentos para Construção Sem Operador', 'Fabricação de Produtos Derivados do Petróleo, exceto Produtos do Refino', 'Fabricação de Medicamentos para Uso Humano# Comércio Atacadista de Produto

In [7]:
def rotular_setor(setor: str, agrupamentos: Dict[str, Any]) -> str:
    """
    Rotula um setor específico baseado nos agrupamentos criados
    """
    for grupo_key, grupo_info in agrupamentos.items():
        if setor in grupo_info.get('setores', []):
            return grupo_info.get('nome', grupo_key)
    
    # Se não encontrou correspondência exata, usa o Gemini para classificar
    prompt = f"""
    Baseado nos seguintes grupos existentes, classifique o setor "{setor}":

    Grupos disponíveis:
    {json.dumps({k: v['nome'] for k, v in agrupamentos.items()}, indent=2, ensure_ascii=False)}

    Retorne apenas o nome do grupo mais apropriado para o setor "{setor}".
    Regras:
    1. Se não houver correspondência clara, responda "Outros"
    2. Não crie novos grupos
    3. Use exatamente os nomes dos grupos fornecidos
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Erro ao rotular setor {setor}: {e}")
        return "Não Classificado"

def classificar_dataframe_setores(df: pd.DataFrame, coluna_setor: str, num_grupos: int = 5) -> pd.DataFrame:
    """
    Função principal que classifica todo o dataframe
    """
    # Obtém setores únicos
    setores_unicos = df[coluna_setor].dropna().unique().tolist()
    
    print(f"Encontrados {len(setores_unicos)} setores únicos")
    print("Criando agrupamentos com Gemini...")
    
    # Cria agrupamentos
    agrupamentos = criar_agrupamentos_setores(setores_unicos, num_grupos)
    
    if not agrupamentos:
        print("Falha ao criar agrupamentos")
        return df
    
    print("Agrupamentos criados:")
    for grupo_key, grupo_info in agrupamentos.items():
        print(f"- {grupo_info.get('nome', grupo_key)}: {len(grupo_info.get('setores', []))} setores")
    
    # Aplica rótulos
    print("Aplicando rótulos...")
    df['grupo_setor'] = df[coluna_setor].apply(lambda x: rotular_setor(x, agrupamentos) if pd.notna(x) else "Não Informado")
    
    return df

In [8]:
classificar_dataframe_setores(df, 'setor_economico', 15)

Encontrados 42 setores únicos
Criando agrupamentos com Gemini...
Agrupamentos criados:
- Indústria Automotiva: 4 setores
- Comércio de Combustíveis: 3 setores
- Construção Civil: 5 setores
- Indústria Química: 7 setores
- Setor de Energia: 6 setores
- Saúde e Farmácia: 5 setores
- Comércio Varejista: 4 setores
- Indústria de Medição: 1 setores
- Serviços de Engenharia: 1 setores
- Telecomunicações: 1 setores
- Transporte de Carga: 2 setores
- Gestão Imobiliária: 2 setores
- Logística Portuária: 1 setores
- Alimentos Preparados: 1 setores
- Representantes Comerciais: 3 setores
Aplicando rótulos...


,id,ano_documento,assinaturas,descricao_tipo_documento,decisao_tribunal,setor_economico,id_unidade,data_processo,data_documento,diferenca_dias,mercado_relevante,descricao_especificacao,conteudo,resumo,grupo_setor
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,Outras,Voto Processo Administrativo,vazio,Fabricação de Peças e Acessórios para o Sistem...,110000967.0,2019-09-13,2024-05-23,1714,"pistões de motor, bronzinas, camisas, pinos, b...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",SEI/CADE - 1390607 - Voto Processo Administrat...,"['\nREPRESENTADOS(AS): BERND BRÜNIG, FAUSTINO ...",Indústria Automotiva
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,condenacao,Gestão e Administração da Propriedade Imobiliária,110000969.0,2015-05-21,2024-05-09,3276,Mercado imobiliário,Influencia de conduta uniforme - COFECI,SEI/CADE - 1384594 - Voto Processo Administrat...,"['\nNOS TERMOS DO VOTO RECORRIDO, ESTE TRIBUNA...",Gestão Imobiliária
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,2024,Outras,Voto,arquivamento,"Comércio Atacadista de Combustíveis Sólidos, L...",110001021.0,2022-08-08,2024-05-09,640,Postos de gasolina,Ministério Público do Estado de Minas Gerais,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,['\nREPRESENTADOS(AS): AUTO POSTO PACAEMBU LTD...,Comércio de Combustíveis
3,c8499708-5778-45d9-8914-9ab97b77c1fd,2024,Alexandre Cordeiro Macedo,Voto Processo Administrativo,condenacao,Construção de Edifícios# Construção de Rodovia...,110000955.0,2016-11-22,2024-04-23,2709,obras de engenharia civil# Construção de infra...,PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377990 - Voto Processo Administrat...,['\nREPRESENTADOS(AS): ANDRADE GUTIERREZ ENGEN...,Construção Civil
4,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,condenacao,Construção de Edifícios# Construção de Rodovia...,110000969.0,2016-11-22,2024-04-23,2709,obras de engenharia civil# Construção de infra...,PAC Favelas - AL 09/2016 - Favelas,SEI/CADE - 1377023 - Voto Processo Administrat...,['\nREPRESENTADOS(AS): ANDRADE GUTIERREZ ENGEN...,Construção Civil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,fe719fd6-f685-4e89-ad9f-943fcff4f602,2021,Paula Farani de Azevedo Silveira,Voto Processo Administrativo,arquivamento,Fabricação de Componentes Eletrônicos,110000970.0,2017-04-19,2021-06-21,1524,"Linha de Assinante Digital Assimétrica - ADSL,...",NaN,SEI/CADE - 0919607 - Voto Processo Administrat...,"['\nNO ÂMBITO DO PROCESSO ORIGINÁRIO, CONSTATO...",Representantes Comerciais
196,2acedd15-07da-4643-a856-5d791e2bd40c,2021,Paula Farani de Azevedo Silveira,Voto Processo Administrativo,condenacao,Gestão de Portos e Terminais,110000970.0,2015-08-31,2021-06-21,2121,Mercado de transporte marítimo de contêineres#...,OGMO BVC,SEI/CADE - 0919624 - Voto Processo Administrat...,['\nREPRESENTADOS(AS): ÓRGÃO GESTOR DE MÃO DE ...,Logística Portuária
197,e8ebd86a-9a51-443f-9236-edd08512196c,2021,Sérgio Costa Ravagnani,Voto Processo Administrativo,arquivamento,Fabricação de Componentes Eletrônicos,110000971.0,2017-04-19,2021-06-21,1524,"Linha de Assinante Digital Assimétrica - ADSL,...",NaN,SEI/CADE - 0920855 - Voto Processo Administrat...,['\nDISPOSITIVO\n-----------------------------...,Representantes Comerciais
198,b9945844-7c20-44b1-a28a-c458f1621cd6,2021,Maurício Oscar Bandeira Maia,Voto Processo Administrativo,arquivamento,"Comércio Varejista de Livros, Jornais, Revista...",110000969.0,2012-10-10,2021-06-21,3176,"Material escolar# Aquisição de uniformes, moch...",Uniformes: Autos Públicos,SEI/CADE - 0921417 - Voto Processo Administrat...,['\nPROCESSO ADMINISTRATIVO. CARTEL. LICITAÇÕE...,Outros


In [9]:
df['grupo_setor'].value_counts()

grupo_setor
Indústria Química            32
Construção Civil             26
Comércio de Combustíveis     24
Indústria Automotiva         21
Setor de Energia             19
Representantes Comerciais    14
Outros                       12
Transporte de Carga          11
Telecomunicações             10
Comércio Varejista           10
Saúde e Farmácia              7
Indústria de Medição          5
Gestão Imobiliária            4
Logística Portuária           3
Alimentos Preparados          2
Name: count, dtype: int64

In [14]:
col = 'grupo_setor'
cols = list(df.columns)
cols.insert(6, cols.pop(cols.index(col)))
df = df[cols]

df.head(3)

,id,ano_documento,assinaturas,descricao_tipo_documento,decisao_tribunal,setor_economico,grupo_setor,id_unidade,data_processo,data_documento,diferenca_dias,mercado_relevante,descricao_especificacao,conteudo,resumo
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,Outras,Voto Processo Administrativo,vazio,Fabricação de Peças e Acessórios para o Sistem...,Indústria Automotiva,110000967.0,2019-09-13,2024-05-23,1714,"pistões de motor, bronzinas, camisas, pinos, b...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",SEI/CADE - 1390607 - Voto Processo Administrat...,"['\nREPRESENTADOS(AS): BERND BRÜNIG, FAUSTINO ..."
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,condenacao,Gestão e Administração da Propriedade Imobiliária,Gestão Imobiliária,110000969.0,2015-05-21,2024-05-09,3276,Mercado imobiliário,Influencia de conduta uniforme - COFECI,SEI/CADE - 1384594 - Voto Processo Administrat...,"['\nNOS TERMOS DO VOTO RECORRIDO, ESTE TRIBUNA..."
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,2024,Outras,Voto,arquivamento,"Comércio Atacadista de Combustíveis Sólidos, L...",Comércio de Combustíveis,110001021.0,2022-08-08,2024-05-09,640,Postos de gasolina,Ministério Público do Estado de Minas Gerais,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,['\nREPRESENTADOS(AS): AUTO POSTO PACAEMBU LTD...


In [15]:
df_final.to_csv("../dados/df_setorizado.csv", index=False)